In [1]:
import pandas as pd
import requests
import json
import datetime as datetime
import matplotlib.pyplot as plt

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
# Requesting the Covid Cases data from our world in data website as a json object.
data_cases=requests.get('https://covid.ourworldindata.org/data/owid-covid-data.json')

# Requesting the vaccination data from our world in data website as a json object:
data_Vax=requests.get('https://github.com/owid/covid-19-data/raw/master/public/data/vaccinations/vaccinations.json')

In [4]:
# load json object for the total number of COVID cases 
json_object_cases=json.loads(data_cases.content)

#Dumping all data from json into a variable:
json_object_vax=json.loads(data_Vax.content)

In [5]:
country_list = [
    'Italy',
    'US',
    'Spain',
    'Germany',
    'Korea, South',
]

# Relative Cases (Absolute Covid Cases/Population Size)


In [6]:
# Obtain the country data for Austrlia from the json dictionary
cases_country_data = json_object_cases['AUS']['data'][:]
#create the list for storing the dates 
list_date=[]

# Browsing over the entire country data
for index in range(len(cases_country_data)):
    # Obtaining the dates from the json dictionary
    date_list = json_object_cases['AUS']['data'][index]
    # append the dates to the date list
    list_date.append(date_list['date'])

# convert from string to date time format
list_date=pd.to_datetime(pd.Series(list_date))

# Create a dataframe for dates in Australia
pd_dates_AUS = pd.DataFrame(list_date)
# rename the column indices to date
pd_dates_AUS = pd_dates_AUS.rename(columns={0:'date'}) 
# Get the date time format in %Y-%m-%d
pd_dates_AUS['date'] = pd.to_datetime(pd_dates_AUS['date'],format='%Y-%m-%d')   

# Get the total population for australia
pop_AUS=json_object_cases['AUS']['population']
# Create an empty to store the cases
list_cases=[]
for index in range(len(cases_country_data)):
    # Entire the entire cases list for australia
    cases_list = json_object_cases['AUS']['data'][index]
    # append the cases and divide by the total population
    list_cases.append((cases_list['total_cases']/pop_AUS))
# Convert the cases into the a dataframe
pd_cases_AUS = pd.DataFrame(list_cases)
# Rename the column index to Australia cases  
pd_cases_AUS = pd_cases_AUS.rename(columns={0:'AUS_cases'})

#Concatenate the dates and cases for australia
date_cases_AUS = pd.concat([pd_dates_AUS,pd_cases_AUS],axis=1)

# Relative Vaccinations (Absolute Vaccination/Population Size)

In [9]:
# create an empty dictionary
dict_from_list = {}

# for loop to convert list of dict
# to dict of list
for item in json_object_vax:
	name = item['country']
	dict_from_list[name] = item

#Separating dictonary of USA from list and adding it to new list:
country_USA=dict_from_list['United States']

#Separating the list that contains the data we need from the USA dictonary:
country_USA_data=country_USA['data']

#Creating an empty dictonary to get data(date, vaccination):
dictionary_of_USA_date_vax={}

#Creating dictonary from the list:
for item in country_USA_data:
	name = item['date']
	dictionary_of_USA_date_vax[name] = item

#Creating an empty list:
inter_list_USA=[]

#Adding all the keys from data dictonary to an empty list:
for key in dictionary_of_USA_date_vax:
	inter_list_USA.append(dictionary_of_USA_date_vax[key])


#Converting list to dataframe:
pd_USA_inter=pd.DataFrame(inter_list_USA)

#Renaming the label total_vaccinations column to 'USA ratio vaccination:population' (data it holds while plotting)
pd_USA_inter = pd_USA_inter.rename(columns={'total_vaccinations':'USA_ratio_vax_pop'})

#Setting the index of the dataframe to date:
pd_USA_inter.set_index('date')

#Separating the data columns that we need for the plot (date, total_vaccinations):
pd_USA_fin= pd_USA_inter.iloc[:, [0,1]]

#Getting the total population of United Kingdom from the above requested json data:
pop_USA=json_object_cases['USA']['population']

#Dividing the vaccinated people by total population to get the ratio:
fin_USA=pd_USA_fin.copy()
fin_USA['USA_ratio_vax_pop']=fin_USA['USA_ratio_vax_pop'].div(pop_USA)

# Dashboard Creation 

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    html.Label('Multi-Select Country'),
    dcc.Dropdown(id='country_drop_down',
                 options=[{
                     'label': 'Italy',
                     'value': 'Italy'
                 }, {
                     'label': 'US',
                     'value': 'US'
                 }, {
                     'label': 'Spain',
                     'value': 'Spain'
                 }, {
                     'label': 'Germany',
                     'value': 'Germany'
                 }, {
                     'label': 'Korea, South',
                     'value': 'Korea, South'
                 }],
                 value=['US', 'Germany'],
                 multi=True),
    dcc.Graph(figure=fig, id='main_window_slope')
])

In [ ]:

@app.callback(Output('main_window_slope', 'figure'),
              [Input('country_drop_down', 'value')])
def update_figure(country_list):

    traces = []
    for each in country_list:
        traces.append(
            dict(x=df_plot.date,
                 y=df_plot[each],
                 mode='markers+lines',
                 opacity=0.9,
                 line_width=2,
                 marker_size=1,
                 name=each))

    return {
        'data':
        traces,
        'layout':
        dict(
            width=1280,
            height=720,
            xaxis_title='Time',
            yaxis_title=
            'Confirmed infected people (source johns hopkins case, log-scale)',
            xaxis={
                'tickangle': -45,
                'nticks': 20,
                'tickfont': dict(size=14, color='#7f7f7f'),
            },
            yaxis={
                'type': 'log',
                'range': '[1.1, 5.5]'
            })
    }

In [ ]:
app.run_server(debug=True, use_reloader=False)